AutoRAG Banner

## Monitor KFP run details

### initialize kfp client

In [23]:
import os
import kfp

kubeflow_endpoint = "https://ds-pipeline-dspa-autox.apps.rosa.rhoai-dev.mfe3.p3.openshiftapps.com"
print(f"Connecting to kfp: {kubeflow_endpoint}")

sa_token_path = os.environ["KF_PIPELINES_SA_TOKEN_PATH"]
sa_ca_cert = os.environ["PIPELINES_SSL_SA_CERTS"]

if "BEARER_TOKEN" in os.environ:
    print("BEARER_TOKEN provided as environment variable")
    bearer_token = os.environ["BEARER_TOKEN"]
elif os.path.isfile(sa_token_path):
    print("BEARER_TOKEN found as service account token")
    with open(sa_token_path) as f:
        bearer_token = f.read().rstrip()

if os.path.isfile(sa_ca_cert) and "svc" in kubeflow_endpoint:
    print("CA Cert found in container")
    ssl_ca_cert = sa_ca_cert
else:
    ssl_ca_cert = None

client = kfp.Client(
    host=kubeflow_endpoint,
    existing_token=bearer_token,
    ssl_ca_cert=ssl_ca_cert,
)

Connecting to kfp: https://ds-pipeline-dspa-autox.apps.rosa.rhoai-dev.mfe3.p3.openshiftapps.com
BEARER_TOKEN found as service account token


/opt/app-root/lib64/python3.12/site-packages/kfp/client/client.py:159: FutureWarning: This client only works with Kubeflow Pipeline v2.0.0-beta.2 and later versions.
  warnings.warn(


### get pipeline details

In [24]:
pipeline_name = "documents-rag-optimization-pipeline"
pipeline_id = client.get_pipeline_id(pipeline_name)
component_names = []

for ver in client.list_pipeline_versions(pipeline_id=pipeline_id).to_dict()["pipeline_versions"]:
    components = ver["pipeline_spec"]["pipeline_spec"]["components"]
    for key, value in components.items():
        component_names.append(key.removeprefix("comp-"))

print("components", set(component_names))

components {'search-space-preparation', 'text-extraction', 'test-data-loader', 'document-loader', 'rag-templates-optimization'}


### list experiments

In [25]:
experiment_name = "Granite code models"
experiment_id = None

for exp in client.list_experiments().to_dict()["experiments"]:
    if exp["display_name"] == experiment_name:
        experiment_id = exp["experiment_id"]
    
print("experiment_id", experiment_id)

experiment_id 5eb08a13-a6d7-42b2-bda8-fdd5118b31f8


### get run details

In [26]:
run_id = "8040d892-57d9-4e17-b97f-6cee2b11bf67"

### check run status

In [27]:
"""
{'run_id': 'd3998ea8-2be4-4f86-8598-85d46b0038c9',
   'task_id': 'b092fef4-11cd-49d3-b3ba-6052bd01f6fd',
   'display_name': 'iteration-item-driver',
   'create_time': datetime.datetime(2026, 1, 27, 9, 50, 29, tzinfo=tzlocal()),
   'start_time': datetime.datetime(2026, 1, 27, 9, 56, 15, tzinfo=tzlocal()),
   'end_time': datetime.datetime(2026, 1, 27, 9, 56, 18, tzinfo=tzlocal()),
   'executor_detail': None,
   'state': 'SUCCEEDED',
   'execution_id': None,
   'error': None,
   'inputs': None,
   'outputs': None,
   'parent_task_id': None,
"""
run = client.get_run(run_id=run_id)
run_details = run.run_details.to_dict()

for task in run_details['task_details']:
    if task["display_name"] and task["display_name"] in component_names:
        print(task["display_name"], task["state"])

text-extraction SUCCEEDED
document-loader SUCCEEDED
rag-templates-optimization SUCCEEDED
test-data-loader SUCCEEDED
search-space-preparation SUCCEEDED


## List all stored assets

In [28]:
import boto3
import os
from IPython.display import HTML

s3 = boto3.resource('s3', endpoint_url=os.environ['AWS_S3_ENDPOINT'])
bucket = s3.Bucket(os.environ['AWS_S3_BUCKET'])
prefix = os.path.join(pipeline_name, run_id)


for obj in bucket.objects.filter(Prefix=prefix):
    print(obj.key)

documents-rag-optimization-pipeline/8040d892-57d9-4e17-b97f-6cee2b11bf67/leaderboard-evaluation/cc01c539-8af6-4ab5-b659-81092c58f529/executor-logs
documents-rag-optimization-pipeline/8040d892-57d9-4e17-b97f-6cee2b11bf67/leaderboard-evaluation/cc01c539-8af6-4ab5-b659-81092c58f529/html_artifact
documents-rag-optimization-pipeline/8040d892-57d9-4e17-b97f-6cee2b11bf67/rag-templates-optimization/015350b7-099f-423c-8846-0cc1b09d06f7/executor-logs
documents-rag-optimization-pipeline/8040d892-57d9-4e17-b97f-6cee2b11bf67/rag-templates-optimization/015350b7-099f-423c-8846-0cc1b09d06f7/rag_patterns/pattern0/evaluation_result.json
documents-rag-optimization-pipeline/8040d892-57d9-4e17-b97f-6cee2b11bf67/rag-templates-optimization/015350b7-099f-423c-8846-0cc1b09d06f7/rag_patterns/pattern0/indexing_notebook.ipynb
documents-rag-optimization-pipeline/8040d892-57d9-4e17-b97f-6cee2b11bf67/rag-templates-optimization/015350b7-099f-423c-8846-0cc1b09d06f7/rag_patterns/pattern0/inference_notebook.ipynb
docume

## RAG Pattern details

In [29]:
import json

for obj in bucket.objects.filter(Prefix=prefix):
    if "pattern.json" in obj.key:
        bucket.download_file(obj.key, "rag_pattern.json")
        break;

with open("rag_pattern.json") as f:
    d = json.load(f)
    print(json.dumps(d, indent=4))

{
    "pattern_name": "pattern0",
    "collection": "collection0",
    "scores": {
        "scores": {
            "answer_correctness": {
                "mean": 0.5,
                "ci_low": 0.4,
                "ci_high": 0.7
            },
            "faithfulness": {
                "mean": 0.2,
                "ci_low": 0.1,
                "ci_high": 0.5
            },
            "context_correctness": {
                "mean": 1.0,
                "ci_low": 0.9,
                "ci_high": 1.0
            }
        },
        "question_scores": {
            "answer_correctness": {
                "q_id_0": 0.5,
                "q_id_1": 0.8
            },
            "faithfulness": {
                "q_id_0": 0.5,
                "q_id_1": 0.8
            },
            "context_correctness": {
                "q_id_0": 1.0,
                "q_id_1": 1.0
            }
        }
    },
    "execution_time": 0.0,
    "final_score": 0.5,
    "schema_version": "1.0",
    "produ

## Evaluation results

In [30]:
eval_filename = "evaluation_result.json"

for obj in bucket.objects.filter(Prefix=prefix):
    if eval_filename in obj.key:
        bucket.download_file(obj.key, eval_filename)
        break

with open(eval_filename) as f:
    d = json.load(f)
    print(json.dumps(d, indent=4))

[
    {
        "question": "benchmark_data.questions[idx]",
        "correct_answers": "benchmark_data.answers[idx]",
        "question_id": "benchmark_data.questions_ids[idx]",
        "answer": "inference_response[idx][\"answer\"]",
        "answer_contexts": [
            {
                "text": "c",
                "document_id": "c"
            },
            {
                "text": "o",
                "document_id": "o"
            },
            {
                "text": "n",
                "document_id": "n"
            },
            {
                "text": "t",
                "document_id": "t"
            },
            {
                "text": "e",
                "document_id": "e"
            },
            {
                "text": "x",
                "document_id": "x"
            },
            {
                "text": "t",
                "document_id": "t"
            },
            {
                "text": "s",
                "document_id": "_"
      

## Display leaderboard

**NOTE:** Make sure the connection has been added to the workbench to access the results S3 secrets.

In [32]:
import boto3
import os
from IPython.display import HTML

s3 = boto3.resource('s3', endpoint_url=os.environ['AWS_S3_ENDPOINT'])
bucket = s3.Bucket(os.environ['AWS_S3_BUCKET'])
prefix = os.path.join(pipeline_name, run_id)
leaderboard_prefix = os.path.join(prefix, 'leaderboard-evaluation')
leaderboard_artifact_name = 'html_artifact'

for obj in bucket.objects.filter(Prefix=prefix):
    if leaderboard_artifact_name in obj.key:
        bucket.download_file(obj.key, leaderboard_artifact_name)

HTML(leaderboard_artifact_name)

Pattern_Name,mean_answer_correctness,mean_faithfulness,mean_context_correctness,chunking.method,chunking.chunk_size,chunking.chunk_overlap,embeddings.model_id,retrieval.method,retrieval.number_of_chunks,generation.model_id
pattern2,0.7000,0.4000,1.0000,recursive,256,128,mock-embed-a,window,5,mock-llm-2
pattern1,0.6000,0.3000,1.0000,recursive,256,128,mock-embed-b,window,5,mock-llm-1
pattern0,0.5000,0.2000,1.0000,recursive,256,128,mock-embed-a,window,5,mock-llm-1


In [10]:
model_to_download = "LightGBMXT_BAG_L2_FULL"

component_full_fit_name = 'autogluon-models-full-refit'
full_refit_prefix = os.path.join(prefix, component_full_fit_name)
best_model_subpath = os.path.join("model_artifact", model_to_download)
best_model_path = None
local_dir=None

for obj in bucket.objects.filter(Prefix=full_refit_prefix):
    if best_model_subpath in obj.key:
        target = obj.key if local_dir is None \
            else os.path.join(local_dir, os.path.relpath(obj.key, s3_folder))
        if not os.path.exists(os.path.dirname(target)):
            os.makedirs(os.path.dirname(target))
        if obj.key[-1] == '/':
            continue
        bucket.download_file(obj.key, target)
        best_model_path = obj.key

best_model_path = os.path.join(best_model_path.split(model_to_download)[0], model_to_download)

---